In [6]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [7]:
from langchain_aws import ChatBedrock  # Bedrock 모델 사용
from langchain_core.messages import HumanMessage, SystemMessage

# Bedrock 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지 템플릿 생성
system_template = "Translate the following into {language}:"

# 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# 템플릿 호출
prompt_result = prompt_template.invoke({"language": "korean", "text": "hi"})
print(prompt_result.to_messages())

[SystemMessage(content='Translate the following into korean:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]


In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [12]:
# 프롬프트, 모델, 파서를 결합한 체인 생성
# chain = prompt_template | model | parser
chain = model | parser

In [14]:
result = chain.stream("Explain AI")
for chunk in result:
    print(chunk, end="")

Artificial Intelligence (AI) is a broad field of computer science focused on creating intelligent machines that can perform tasks that typically require human intelligence. Here's a brief overview of AI:

1. Definition: AI refers to systems or machines that mimic human intelligence to perform tasks and can iteratively improve themselves based on the information they collect.

2. Types of AI:
   - Narrow AI: Designed for specific tasks (e.g., virtual assistants, image recognition)
   - General AI: Hypothetical AI with human-like intelligence across various domains
   - Super AI: Theoretical AI surpassing human intelligence in all areas

3. Key components:
   - Machine Learning: Algorithms that improve through experience
   - Deep Learning: Advanced machine learning using neural networks
   - Natural Language Processing: Enabling computers to understand and generate human language
   - Computer Vision: Allowing machines to interpret and understand visual information

4. Applications:
   

In [15]:
result = chain.invoke("Explain AI")
print(result)

Artificial Intelligence (AI) is a branch of computer science that aims to create intelligent machines capable of performing tasks that typically require human intelligence. These tasks include learning, problem-solving, perception, language understanding, reasoning, and planning.

Key aspects of AI include:

1. Machine Learning: The ability of AI systems to improve their performance on a specific task through experience, without being explicitly programmed.

2. Deep Learning: A subset of machine learning that uses artificial neural networks to model and process complex patterns in data.

3. Natural Language Processing (NLP): The ability of AI to understand, interpret, and generate human language.

4. Computer Vision: The ability to analyze and understand visual information from the world.

5. Robotics: The integration of AI with physical machines to perform tasks in the real world.

6. Expert Systems: AI programs designed to solve complex problems by reasoning through bodies of knowled

In [16]:
inputs = ["What is AI?", "What is Machine Learning?"]
results = chain.batch(inputs=inputs)
print(results)

['AI, or Artificial Intelligence, refers to the development of computer systems that can perform tasks that typically require human intelligence. These tasks include:\n\n1. Learning and problem-solving\n2. Visual perception\n3. Speech recognition\n4. Decision-making\n5. Language translation\n\nAI systems are designed to mimic human cognitive functions and can improve their performance over time through experience and data analysis. There are two main types of AI:\n\n1. Narrow or Weak AI: Designed for a specific task or range of tasks (e.g., virtual assistants, image recognition)\n\n2. General or Strong AI: Hypothetical AI with human-like consciousness and ability to perform any intellectual task\n\nAI technologies include machine learning, deep learning, neural networks, and natural language processing. These are used in various applications such as:\n\n- Autonomous vehicles\n- Healthcare diagnostics\n- Financial trading\n- Personalized recommendations\n- Robotics\n\nAs AI continues to

In [20]:
async def async_stream_example(texts):
    for text in texts:
        async for chunk in chain.astream(input=text):
            print(chunk)

In [23]:
await async_stream_example(["What is AI?", "What is Machine Learning?", "Say Hi"])


AI
,
 or
 Artificial Intelligence, refers
 to the simulation
 of human
 intelligence in
 machines that
 are
 program
med to think an
d learn
 like humans. Key
 aspects
 of AI include:


1
. Machine
 Learning: The
 ability of AI
 systems to learn from
 data
 and improve
 their performance over time
 without
 explicit
 programming.


2. Natural
 Language Processing (N
LP): Enabling
 machines to understand,
 interpret
, and generate human
 language.


3. Computer
 Vision: Allowing
 machines to interpret
 and understand visual information
 from
 the
 world.


4.
 Robotics: The
 integration of AI into
 physical machines
 to
 perform
 tasks.


5. Expert
 Systems: AI
 programs
 designe
d to solve
 complex
 problems by
 mim
icking human expertise
 in
 specific
 domains.

6
. Neural
 Networks: Computational
 models inspired by the
 human brain's
 structure
 and function
.

7.
 Deep
 Learning: A
 subset
 of machine learning that
 uses
 multi
-lay
ered neural networks to
 analyze
 various
 facto

## Runnable

##### Runnable 인터페이스 코드 구현

In [25]:
# Runnable 인터페이스 정의
class Runnable:
    def invoke(self, input):
        raise NotImplementedError("The 'invoke' method must be implemented!")

    def stream(self, input):
        raise NotImplementedError("The 'stream' method must be implemented!")

In [26]:
# Runnable 인터페이스를 상속받은 클래스
class ChatModel(Runnable):
    def invoke(self, input):
        return f"ChatModel response to: {input}"

# stream 메서드가 구현되지 않음
chat_model = ChatModel()

# invoke 메서드는 정상적으로 작동
print(chat_model.invoke("Tell me about AI"))

# stream 메서드는 구현되지 않았으므로 에러 발생
try:
    for chunk in chat_model.stream("Tell me about AI"):
        print(chunk)
except NotImplementedError as e:
    print(f"Error: {e}")

ChatModel response to: Tell me about AI
Error: The 'stream' method must be implemented!


In [27]:
# 모든 메서드를 구현한 ChatModel 클래스
class ChatModel(Runnable):
    def invoke(self, input):
        return f"ChatModel response to: {input}"

    def stream(self, input):
        for word in input.split():
            yield f"Streaming: {word}"

# 이번에는 모든 메서드가 구현되었으므로 에러가 발생하지 않음
chat_model = ChatModel()

# 정상적으로 invoke와 stream 호출
print(chat_model.invoke("Tell me about AI"))
for chunk in chat_model.stream("Tell me about AI"):
    print(chunk)

ChatModel response to: Tell me about AI
Streaming: Tell
Streaming: me
Streaming: about
Streaming: AI


### 채팅 모델에서 Runnable 구현

In [28]:
class ChatModel(Runnable):
    def invoke(self, input):
        # 채팅 모델이 입력에 대한 응답을 생성
        response = f"Response to {input}"
        return response

    def stream(self, input):
        # 입력에 대해 응답을 스트리밍
        for word in input.split():
            yield f"Streaming: {word}"

# Runnable 인터페이스를 구현한 ChatModel 호출
chat_model = ChatModel()
result = chat_model.invoke("Tell me about AI")
for chunk in chat_model.stream("Tell me about AI"):
    print(chunk)

Streaming: Tell
Streaming: me
Streaming: about
Streaming: AI


### 출력 파서에서 Runnable 구현

In [29]:
class OutputParser(Runnable):
    def invoke(self, input):
        # LLM 또는 채팅 모델의 출력을 파싱하여 구조화된 데이터 반환
        return {"parsed_output": input}

# Runnable 인터페이스를 구현한 OutputParser 호출
parser = OutputParser()
parsed_result = parser.invoke("This is a raw output")
print(parsed_result)

{'parsed_output': 'This is a raw output'}


### 리트리버에서 Runnable 구현

In [30]:
class Retriever(Runnable):
    def invoke(self, query):
        # 데이터베이스 또는 문서에서 쿼리에 맞는 정보를 검색
        return [f"Document related to: {query}"]

# Runnable 인터페이스를 구현한 Retriever 호출
retriever = Retriever()
documents = retriever.invoke("Explain AI")
print(documents)

['Document related to: Explain AI']


### 프롬프트에서 Runnable 구현

In [31]:
class PromptTemplate(Runnable):
    def invoke(self, input):
        # 입력을 기반으로 프롬프트 생성
        return f"Generated prompt: {input}"

# Runnable 인터페이스를 구현한 PromptTemplate 호출
template = PromptTemplate()
prompt = template.invoke("Tell me about AI")
print(prompt)

Generated prompt: Tell me about AI


---

In [ ]:
from langchain.chat_models import ChatOpenAI

# Chat 모델 인스턴스 생성
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", api_key="your-api-key")

# 메시지 기반 대화
response = chat_model.invoke(input="What is the capital of France?")
print(response)

---

1. Human Message

In [33]:
from langchain_core.messages import HumanMessage

# 사용자 메시지 생성
message = HumanMessage(content="What is the capital of France?")

2. AIMessage

In [34]:
from langchain_core.messages import AIMessage

# AI 메시지 생성
ai_message = AIMessage(content="The capital of France is Paris.", response_metadata={"logprobs": [0.1, 0.9]})

3. SystemMessage

In [ ]:
from langchain_core.messages import SystemMessage

# 시스템 메시지 생성
system_message = SystemMessage(content="You are a helpful assistant.")

4. ToolMessage

In [36]:
from langchain_core.messages import ToolMessage

# 도구 호출 결과 메시지 생성
tool_message = ToolMessage(content="The capital of France is Paris.", tool_call_id="12345", artifact={"file": "output.csv"})

---

### Prommpt Template의 유형

1. String PromptTemplates

In [38]:
from langchain_core.prompts import PromptTemplate

# 주제에 맞는 농담 생성 프롬프트
prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

# 주제 'cats'로 프롬프트 실행
response = prompt_template.invoke({"topic": "cats"})
print(response)  # "Tell me a joke about cats"

text='Tell me a joke about cats'


2. ChatPromptTemplates

In [44]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지와 사용자 메시지를 포함한 프롬프트
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

# 주제 'cats'로 프롬프트 실행
response = prompt_template.invoke({"topic": "cats"})
print(response)  # ["You are a helpful assistant", "Tell me a joke about cats"]

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})]


3. MessagesPlaceholder

In [40]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# 메시지 플레이스홀더 사용한 템플릿
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

# 메시지 리스트를 입력으로 전달
response = prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})
print(response)  # ["You are a helpful assistant", "hi!"]

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


---

### Example Selector
##### Few-Shot Prompting

In [47]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

In [51]:
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivaderci"},
    {"input": "soccer", "output": "calcio"},
]

In [50]:
### Custom Example Selector
from langchain_core.example_selectors.base import BaseExampleSelector

class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # This assumes knowledge that part of the input will be a 'text' key
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        # Initialize variables to store the best match and its length difference
        best_match = None
        smallest_diff = float("inf")

        # Iterate through each example
        for example in self.examples:
            # Calculate the length difference with the first word of the example
            current_diff = abs(len(example["input"]) - new_word_length)

            # Update the best match if the current one is closer in length
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

In [52]:
example_selector = CustomExampleSelector(examples)

In [53]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italian:",
    input_variables=["input"],
)

print(prompt.format(input="word"))

Translate the following words from English to Italian:

Input: bye -> Output: arrivaderci

Input: word -> Output:


In [54]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italain:",
    input_variables=["input"],
)

print(prompt.format(input="word"))

Translate the following words from English to Italain:

Input: bye -> Output: arrivaderci

Input: word -> Output:


---

### Output Parser
1. JSON Output Parser

In [55]:
from langchain_core.output_parsers import JsonOutputParser

# JSON 출력 파서 사용
output_parser = JsonOutputParser()

# 예시 출력 텍스트
output = '{"name": "Alice", "age": 30}'

# JSON 객체로 변환
parsed_output = output_parser.parse(output)
print(parsed_output)  # {"name": "Alice", "age": 30}

{'name': 'Alice', 'age': 30}


2. XML Output Parser

In [58]:
from langchain_core.output_parsers import XMLOutputParser

# XML 출력 파서 사용
output_parser = XMLOutputParser()

# 예시 출력 텍스트
output = '<person><name>Alice</name><age>30</age></person>'

# XML 사전으로 변환
parsed_output = output_parser.parse(output)
print(parsed_output)  # {"person": {"name": "Alice", "age": 30}}

{'person': [{'name': 'Alice'}, {'age': '30'}]}


---

### Chat History

In [59]:
from langchain.memory import ChatMessageHistory

# ChatHistory 인스턴스 생성
history = ChatMessageHistory()

# 대화 기록 추가
history.add_user_message("What is AI?")
history.add_ai_message("AI stands for Artificial Intelligence.")

# 대화 기록 불러오기
messages = history.messages
for message in messages:
    print(message.content)

What is AI?
AI stands for Artificial Intelligence.


---

### Document 객체

In [60]:
from langchain.schema import Document

# Document 객체 생성
doc = Document(page_content="This is the content of the document.", metadata={"id": 123, "file_name": "example.txt"})

# 내용과 메타데이터 출력
print(doc.page_content)  # 문서 내용
print(doc.metadata)      # 문서 메타데이터

This is the content of the document.
{'id': 123, 'file_name': 'example.txt'}


### Document Loaders

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSVLoader 인스턴스 생성
loader = CSVLoader(
    file_path="example.csv"  # CSV 파일 경로
)

# CSV 파일 로드하여 Document 객체로 반환
documents = loader.load()

# 로드된 문서 출력
for doc in documents:
    print(doc.page_content)
    print(doc.metadata)

---

### Text Splitters

1. Character Text Splitter

In [62]:
from langchain.text_splitter import CharacterTextSplitter

# CharacterTextSplitter 설정
text_splitter = CharacterTextSplitter(separator='', chunk_size=50, chunk_overlap=10)

# 예시 문서 텍스트
text = "This is a long document that needs to be split into smaller chunks for processing."

# 텍스트를 청크로 분할
chunks = text_splitter.split_text(text)

for chunk in chunks:
    print(chunk)

This is a long document that needs to be split int
split into smaller chunks for processing.


2. Recursive Text Splitter

In [64]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# CharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)

# 예시 문서 텍스트
text = "This is a long document that needs to be split into smaller chunks for processing."

# 텍스트를 청크로 분할
chunks = text_splitter.split_text(text)

for chunk in chunks:
    print(chunk)

This is a long document that needs to be split
be split into smaller chunks for processing.
